In [1]:
import backtrader as bt
import pandas as pd
import datetime
import backtrader.feeds as btfeeds
from IPython.core.interactiveshell import InteractiveShell 
import pyecharts.options as opts
from pyecharts.charts import Grid,Line,Page,Scatter,Page,Bar
from pyecharts.faker import Faker
InteractiveShell.ast_node_interactivity = "all"
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
import numpy as np
from collections import deque
from tabulate import tabulate
st_date = datetime.datetime(2023, 5, 4)
ed_date = datetime.datetime(2023, 5, 24)

# tf = pd.read_csv('tradeinfo.csv',parse_dates = True,infer_datetime_format=True,header=None)
# tf.columns = ['datetime','dir','ct','price','lot','dd','tp','info','g','ddd']
# tf = tf.set_index('datetime')
# tf.index = pd.to_datetime(tf.index,format='%Y-%m-%d_%H:%M:%S.%f')

# tf = tf.loc['2023-4-27':'2023-05-27']
# tradelist = []
# for idx,r in tf.iterrows():
#     # print(r)
#     ct = 'CN'
#     # print(r['dir'])
#     if 'MCAF' in r['ct']:
#         ct = 'MCAF'
#     size = r['lot'] if r['dir'] == 'buy' else -r['lot']
#     tradelist.append(
#                         (
#                         idx,#datetime.datetime.strftime(idx,'%Y-%m-%d %H:%M:%S'),
#                         size,
#                         r['price'],
#                         ct
#                         )
#                     )

In [2]:
mcaf_codes = [
            # 'MCAF2201','MCAF2202','MCAF2203','MCAF2204','MCAF2205','MCAF2206',
            # 'MCAF2207','MCAF2208','MCAF2209','MCAF2210','MCAF2211','MCAF2212',
            'MCAF2301','MCAF2302','MCAF2303','MCAF2304','MCAF2305','MCAF2306'
            ]
cn_codes =  [
            # 'CN2201','CN2202','CN2203','CN2204','CN2205','CN2206',
            # 'CN2207','CN2208','CN2209','CN2210','CN2211','CN2212',
            'CN2301','CN2302','CN2303', 'CN2304','CN2305','CN2306'
            ]

mcaf_data = {}
cn_data = {}

for a in mcaf_codes:
    mcaf_data[a] =  pd.read_csv('Data/MCAF/'+a+'.csv', index_col = 0,skiprows = [0,2],parse_dates=True,encoding='ISO-8859-1').dropna()
for a in cn_codes:
    cn_data[a] =  pd.read_csv('Data/CN/'+a+'.csv', index_col = 0,skiprows = [0,1,3],parse_dates=True,encoding='ISO-8859-1').dropna()

msci = pd.read_csv('Data/MSCI.csv', index_col = 0,skiprows = [0],parse_dates=True,encoding='ISO-8859-1')
a50 = pd.read_csv('Data/A50.csv',index_col = 0,skiprows = [0,1],parse_dates=True,encoding='ISO-8859-1')

In [3]:
cn_data

{'CN2301':                      13076.0000  13080.0000  13076.0000.1  13080.0000.1  \
 2022-12-20 09:00                                                          
 2022-12-20 09:02:00     13056.0     13056.0       13051.0       13053.0   
 2022-12-20 09:03:00     13038.0     13042.0       13036.0       13042.0   
 2022-12-20 09:04:00     13043.0     13043.0       13035.0       13037.0   
 2022-12-20 09:05:00     13034.0     13034.0       13023.0       13025.0   
 2022-12-20 09:06:00     13025.0     13032.0       13025.0       13030.0   
 ...                         ...         ...           ...           ...   
 2023-01-30 16:26:00     14065.0     14065.0       14065.0       14065.0   
 2023-01-30 16:27:00     14066.0     14066.0       14066.0       14066.0   
 2023-01-30 16:28:00     14066.0     14066.0       14065.0       14066.0   
 2023-01-30 16:29:00     14066.0     14066.0       14065.0       14065.0   
 2023-01-30 16:35:00     14065.0     14065.0       14065.0       14065.0   
 


In [4]:
mcaf_feeds = []
cn_feeds = []

mcaf_rolldate = []
cn_rolldate = []

for k in mcaf_data.keys():
    if mcaf_data[k].index[-1] < pd.to_datetime(st_date):
        # print(mcaf_data[k].index[-1])
        continue
    mcaf_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    mcaf_data[k]['Volume'] = mcaf_data[k]['Volume']*100000
    mcaf_data[k] = mcaf_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    mcaf_data[k] = mcaf_data[k].fillna(method= 'ffill').sort_index().drop_duplicates()
    mcaf_data[k] = mcaf_data[k][mcaf_data[k].index.isin(a50.index)]
    mcaf_feeds.append(bt.feeds.PandasData(dataname = mcaf_data[k],name = k))
    mcaf_rolldate.append(mcaf_data[k].resample('1d').last().index[-2]+datetime.timedelta(hours=14,minutes=57))



msci = msci.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index()
msci.fillna(inplace = True,method = 'ffill')

a50 = a50.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index()
a50.fillna(inplace = True,method = 'ffill')

for k in cn_data.keys():    
    # cn_data[k]
    if cn_data[k].index[-1] < pd.to_datetime(st_date):
        # print(cn_data[k].index[-1])
        continue

    cn_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    cn_data[k]['Volume'] = cn_data[k]['Volume']*100000
    cn_data[k] = cn_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    cn_data[k] = cn_data[k].fillna(method= 'ffill')
    cn_data[k] = cn_data[k].loc[cn_data[k].index&a50.index].dropna().sort_index().drop_duplicates()
    cn_data[k]=cn_data[k][cn_data[k].index.isin(a50.index)]
    cn_feeds.append(bt.feeds.PandasData(dataname = cn_data[k],name = k))
    cn_rolldate.append(cn_data[k].resample('1d').last().index[-1]+datetime.timedelta(hours=14,minutes=57))

msci_feed = bt.feeds.PandasData(dataname = msci, fromdate=st_date, todate=ed_date)

a50_feed = bt.feeds.PandasData(dataname = a50, fromdate=st_date, todate=ed_date)

mcaf_rolldate = mcaf_rolldate[:-1]
cn_rolldate = cn_rolldate[:-1]

def cn_expire(dt,d):
    return dt in cn_rolldate

def mcaf_expire(dt,d):
    if dt in mcaf_rolldate :
        return True
    return False

C:\Users\siruo\AppData\Local\Temp\ipykernel_2256\2846411047.py:37: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cn_data[k] = cn_data[k].loc[cn_data[k].index&a50.index].dropna().sort_index().drop_duplicates()
C:\Users\siruo\AppData\Local\Temp\ipykernel_2256\2846411047.py:37: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cn_data[k] = cn_data[k].loc[cn_data[k].index&a50.index].dropna().sort_index().drop_duplicates()


In [5]:
cn_data['CN2302'][cn_data['CN2302'].index.isin(a50.index)]

,13565,13565.1,13558,13558.1,12,2557
2023/1/10 9:00,,,,,,


In [6]:
class MCAFStrategy(bt.Strategy):
    params = (

        ('margin', 4),  
        ('distor', 1),
        ('bias', 0),
        ('mcaf_lot',1),
        ('cn_lot',4),
        ('logname','log.csv'),

    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.msci_close[0],
                                            self.a50_close[0],
                                            self.mcaf_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('MCAF').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    

    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                       
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Datetime,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,MSCI,A50,MCAF,CN,MCAF_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.mcaf_close = self.data0.close
        self.cn_close = self.data1.close
        self.msci_close = self.data2.close
        self.a50_close = self.data3.close
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        
    def next(self):
        
        self.cn_theo = self.a50_close[0]*(self.p.mcaf_lot*self.mcaf_close[0])/(self.p.mcaf_lot*(self.msci_close[0]))
        # print(self.a50_close[0],self.csi300_close[0],self.csi500_close[0],self.if_close[0],self.ic_close[0])
        cn_bid = self.cn_theo - self.p.margin + self.base
        cn_ask = self.cn_theo + self.p.margin + self.base
        self.log("%.2f,%.2f,%.2f,%.d,%.d"%(self.mcaf_close[0],self.cn_close[0],self.msci_close[0],self.a50_close[0],self.broker.get_value()))
        self.write_record()
        if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
            size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
            self.buy( data = self.getdatabyname('MCAF'),
                      size = self.p.mcaf_lot*size,
                      exectype=bt.Order.Market,coc=True)
            self.sell( data = self.getdatabyname('CN'),
                       size = self.p.cn_lot*size, # size < 0
                       exectype=bt.Order.Market,coc=True)
            self.base = self.base + size * self.p.distor # base 上移，抬高理论价
            
        if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
            size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
            self.sell( data = self.getdatabyname('MCAF'),
                       size = self.p.mcaf_lot*size,
                       exectype=bt.Order.Market,coc=True)
            self.buy( data = self.getdatabyname('CN'),
                      size = self.p.cn_lot*size, # size < 0
                      exectype=bt.Order.Market,coc=True)
            self.base = self.base - size * self.p.distor

        if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in mcaf_rolldate:
            print('ROLL')
            self.base = 0

            if self.getpositionbyname('MCAF').size != 0:
                self.order_target_size( data = self.getdatabyname('MCAF'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('CN').size != 0:
                self.order_target_size( data = self.getdatabyname('CN'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
        if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
            print('ROLL')
            self.base = 0
            if self.getpositionbyname('MCAF').size != 0:
                self.order_target_size( data = self.getdatabyname('MCAF'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('CN').size != 0:
                self.order_target_size( data = self.getdatabyname('CN'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)

    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [7]:
from tabulate import tabulate

cerebro = bt.Cerebro(cheat_on_open = True)
cerebro.broker.set_coc(True)
cerebro.broker.set_cash(1000000)
cerebro.broker.set_shortcash(False)

cerebro.rolloverdata(name='MCAF', *mcaf_feeds, checkdate = mcaf_expire, fromdate=st_date, todate=ed_date)
cerebro.rolloverdata(name='CN', *cn_feeds, checkdate = cn_expire, fromdate=st_date, todate=ed_date)
cerebro.adddata(msci_feed)
cerebro.adddata(a50_feed)

cerebro.broker.setcommission(commission=2.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=25,
                             margin=4000,
                             name='MCAF')
#cerebro.broker.setcommission(commission=30,
                             #commtype=bt.CommInfoBase.COMM_FIXED,
                             #mult=200,
                             #margin=160000,
                             #name='IC')
cerebro.broker.setcommission(commission=1.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=1,
                             margin=1000,
                             name='CN')


cerebro.addstrategy(MCAFStrategy, mcaf_lot = 1, cn_lot = 4, logname = 'MCAF_4CN_Basis.csv')



strats = cerebro.run(runonce = False, cheat_on_open = True, tradehistory = True)


0

2023-05-04T09:30:00, 2153.00,13152.00,2150.14,13137,1000000
2023-05-04T09:31:00, 2149.00,13134.00,2147.58,13130,1000000
2023-05-04T09:32:00, SELL EXECUTED,MCAF,2149.00,-1,0.00,-1,2,1,999996.50

2023-05-04T09:32:00, BUY EXECUTED,CN,13134.00,4,0.00,4,6,2,999996.50

2023-05-04T09:32:00, 2147.20,13124.00,2143.88,13115,999996
2023-05-04T09:33:00, SELL EXECUTED,MCAF,2147.20,-7,0.00,-8,17,3,999673.00

2023-05-04T09:33:00, BUY EXECUTED,CN,13124.00,28,0.00,32,42,4,999673.00

2023-05-04T09:33:00, 2145.00,13102.00,2143.41,13104,999673
2023-05-04T09:34:00, 2144.60,13105.00,2142.83,13094,999849
2023-05-04T09:35:00, BUY EXECUTED,MCAF,2144.60,5,353.12,-3,12,5,999797.50

2023-05-04T09:35:00, SELL EXECUTED,CN,13105.00,-20,-405.00,12,30,6,999797.50

2023-05-04T09:35:00, 2148.40,13128.00,2145.58,13120,999797
2023-05-04T09:36:00, SELL EXECUTED,MCAF,2148.40,-3,0.00,-6,7,7,999760.00

2023-05-04T09:36:00, BUY EXECUTED,CN,13128.00,12,0.00,24,18,8,999760.00

2023-05-04T09:36:00, 2148.00,13125.00,2146.10,13114,

In [8]:
port_record_set = {}
# port_record = pd.read_csv('portlog/3MCAF_28CN_DailyO.csv')
# port_record = port_record[['Datetime','MSCI','A50','MCAF','CN','MCAF_pos','CN_pos','Cash','Value']]
# port_record.set_index('Datetime',inplace=True)
# port_record = port_record.dropna()
# port_record.index = pd.to_datetime(port_record.index)

# port_record_set['DailyOpen'] = port_record

# port_record = pd.read_csv('portlog/replay.csv')
# port_record = port_record[['Datetime','MSCI','A50','MCAF','CN','MCAF_pos','CN_pos','Cash','Value']]
# port_record.set_index('Datetime',inplace=True)
# port_record = port_record.dropna()
# port_record.index = pd.to_datetime(port_record.index)

# port_record_set['Replay'] = port_record
# port_record = pd.read_csv('portlog/3IF_IC_27CN_ABS_ROLLN_JUN_10.csv')
# port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
# port_record.set_index('Datetime',inplace=True)
# port_record = port_record.dropna()
# port_record.index = pd.to_datetime(port_record.index)

# port_record_set['RollN'] = port_record


# abs_port_record = pd.read_csv('portlog/3IF_IC_27CN_ABS_DailyC_JUN_10.csv')
# abs_port_record = abs_port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
# abs_port_record.set_index('Datetime',inplace=True)
# abs_port_record = abs_port_record.dropna()
# abs_port_record.index = pd.to_datetime(abs_port_record.index)

# port_record_set['DailyClose'] = abs_port_record

# port_record = pd.read_csv('portlog/3MCAF_28CN_Basis.csv')
# port_record = port_record[['Datetime','MSCI','A50','MCAF','CN','MCAF_pos','CN_pos','Cash','Value']]
# port_record.set_index('Datetime',inplace=True)
# port_record = port_record.dropna()
# port_record.index = pd.to_datetime(port_record.index)

# port_record_set['Basis'] = port_record



# port_record = pd.read_csv('portlog/3MCAF_28CN_Roll.csv')
# port_record = port_record[['Datetime','MSCI','A50','MCAF','CN','MCAF_pos','CN_pos','Cash','Value']]
# port_record.set_index('Datetime',inplace=True)
# port_record = port_record.dropna()
# port_record.index = pd.to_datetime(port_record.index)

# port_record_set['Roll'] = port_record

In [9]:
port_record_set = {}

port_record = pd.read_csv('portlog/MCAF_4CN_Basis.csv')
port_record = port_record[['Datetime','MSCI','A50','MCAF','CN','MCAF_pos','CN_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['Basis'] = port_record



# port_record = pd.read_csv('portlog/3MCAF_28CN_Roll.csv')
# port_record = port_record[['Datetime','MSCI','A50','MCAF','CN','MCAF_pos','CN_pos','Cash','Value']]
# port_record.set_index('Datetime',inplace=True)
# port_record = port_record.dropna()
# port_record.index = pd.to_datetime(port_record.index)

# port_record_set['Roll'] = port_record

In [10]:
mcaf_rolldate

[Timestamp('2023-05-18 14:57:00', freq='D')]

In [11]:
port_record_set['Basis']

,MSCI,A50,MCAF,CN,MCAF_pos,CN_pos,Cash,Value
Datetime,,,,,,,,
2023-05-04 09:30:00,2150.1,13137.9,2153.0,13152.0,0,0,1000000,1000000
2023-05-04 09:31:00,2147.6,13130.3,2149.0,13134.0,0,0,1000000,1000000
2023-05-04 09:32:00,2143.9,13115.0,2147.2,13124.0,-1,4,991996,999996
2023-05-04 09:33:00,2143.4,13104.0,2145.0,13102.0,-8,32,935673,999673
2023-05-04 09:34:00,2142.8,13094.0,2144.6,13105.0,-8,32,935849,999849
...,...,...,...,...,...,...,...,...
2023-05-23 14:53:00,2100.9,12925.4,2088.8,12909.0,59,-236,531603,1003603
2023-05-23 14:54:00,2100.9,12926.0,2089.2,12909.0,59,-236,532193,1004193
2023-05-23 14:55:00,2100.1,12921.9,2088.0,12903.0,59,-236,531839,1003839


In [12]:
# port_record = port_record.resample('5min').last().dropna()
totalPortValue = (
        Line(
                init_opts=opts.InitOpts(
                width="1440px",
                height="500px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
        )
        .add_xaxis([a.isoformat(sep=' ') for a in port_record_set['Basis'].index.to_list()])
        .add_yaxis(
                    series_name="基差",
                    y_axis=port_record_set['Basis'].Value-port_record_set['Basis'].Value[0],
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width =1)
                    )
        .extend_axis(
                    yaxis=opts.AxisOpts(
                                name = 'CN Pos',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                splitline_opts=opts.SplitLineOpts(  is_show=False,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.2),)
                            ),
                    )
        .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            title_opts = opts.TitleOpts('3MCAF-27CN',),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Value',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                axislabel_opts=opts.LabelOpts(formatter="{value} W"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.2),)
                            ),
                            datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                                # opts.DataZoomOpts(
                                #     is_show=True, type_="slider",orient= "vertical",yaxis_index=[0, 0,1],range_start= -10,range_end=110
                                # ),
                            ],
                             tooltip_opts=opts.TooltipOpts(
                                trigger="axis",
                                axis_pointer_type="cross",
                                background_color="rgba(245, 245, 245, 0.8)",
                                border_width=1,
                                border_color="#ccc",
                                textstyle_opts=opts.TextStyleOpts(color="#000"),
                            ),
                            axispointer_opts=opts.AxisPointerOpts(
                                is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
                            ),
                        )
    )
totalCashUsage = (
    Bar()
    .add_xaxis([a.isoformat(sep=' ') for a in port_record.index.to_list()])
    .add_yaxis(
                series_name="Cash",
                y_axis=port_record_set['Basis'].CN_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.15, )
    )
    .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'position',
                                # max_ = 200,
                                # min_ = 0,
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),

    )
)

totalPortValue.overlap(totalCashUsage)
totalPortValue.render('4cn1mcaf.html')

'c:\\Users\\siruo\\Downloads\\main2\\4cn1mcaf.html'

In [37]:
port_record_set['Basis']

,MSCI,A50,MCAF,CN,MCAF_pos,CN_pos,Cash,Value
Datetime,,,,,,,,
2023-05-04 09:30:00,2150.1,13137.9,2153.0,13152.0,0,0,1000000,1000000
2023-05-04 09:31:00,2147.6,13130.3,2149.0,13134.0,0,0,1000000,1000000
2023-05-04 09:32:00,2143.9,13115.0,2147.2,13124.0,-1,4,991996,999996
2023-05-04 09:33:00,2143.4,13104.0,2145.0,13102.0,-8,32,935673,999673
2023-05-04 09:34:00,2142.8,13094.0,2144.6,13105.0,-8,32,935849,999849
...,...,...,...,...,...,...,...,...
2023-05-23 14:53:00,2100.9,12925.4,2088.8,12909.0,59,-236,531603,1003603
2023-05-23 14:54:00,2100.9,12926.0,2089.2,12909.0,59,-236,532193,1004193
2023-05-23 14:55:00,2100.1,12921.9,2088.0,12903.0,59,-236,531839,1003839
